In [1]:
%cd '..'

/mnt/scratch/v_kulikov/histonedb_Vladimir/CURATED_SET


In [2]:
import inspect
import os
import re
import sys

import pandas as pd
from Bio import Entrez, SeqIO
from curated_set_services import CuratedSet

In [3]:
# Load data from histones.csv
curated_set = CuratedSet()
cs = curated_set

cs.data.shape, cs.data.columns

((566, 16),
 Index(['accession', 'type', 'variant_group', 'variant', 'doublet', 'gi',
        'ncbi_gene_id', 'hgnc_gene_name', 'taxonomy_id', 'organism', 'phylum',
        'class', 'taxonomy_group', 'info', 'references', 'sequence'],
       dtype='object'))

In [4]:
list(cs.has_duplicates())

[]

In [5]:
cs = cs.data[cs.data["variant"] != "late_H2B.1"]
cs.data[cs.data["variant"] != "late_H2B.3"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
NP_505463.1,NP_505463.1,H2A,cH2A,cH2A_(Animals),,17562014,,,6239,Caenorhabditis elegans,Nematoda,Chromadorea,,,26989147 22650316(?),MSGRGKGGKAKTGGKAKSRSSRAGLQFPVGRLHRILRKGNYAQRVG...
EEC09557.1,EEC09557.1,H2A,cH2A,cH2A_(Animals),,215500063,,,6945,Ixodes scapularis,Arthropoda,Arachnida,,,26989147 22650316(?),MSGRGKGGKVKGKSKTRSSRAGLQFPVGRIHRLLRKGNYAERVGAG...
NP_724343.1,NP_724343.1,H2A,cH2A,cH2A_(Animals),,24585673,,,7227,Drosophila melanogaster,Arthropoda,Insecta,,,26989147 22650316(?),MSGRGKGGKVKGKAKSRSNRAGLQFPVGRIHRLLRKGNYAERVGAG...
XP_001119899.1,XP_001119899.1,H2A,cH2A,cH2A_(Animals),,110764935,,,7460,Apis mellifera,Arthropoda,Insecta,,,26989147 22650316(?),MSGRGKGGKAKAKAKSRSNRAGLQFPVGRIHRLLRKGNYAERVGAG...
EDO48405.1,EDO48405.1,H2A,cH2A,cH2A_(Animals),,156227602,,,45351,Nematostella vectensis,Cnidaria,Anthozoa,,,26989147 22650316(?),MSGRGKGKAKGTKSKTRSSRAGLQFPVGRIHRHLRKGNYAERVGAG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NP_005310.1,NP_005310.1,H1,H1.2,H1.2_(Homo_sapiens)__???,,,3006.0,H1-2,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MSETAPAAPAAAPPAEKAPVKKKAAKKAGGTPRKASGPPVSELITK...
NP_005313.1,NP_005313.1,H1,H1.5,H1.5_(Homo_sapiens)__???,,,3009.0,H1-5,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MSETAPAETATPAPVEKSPAKKKATKKAAGAGAAKRKATGPPVSEL...
NP_722575.1,NP_722575.1,H1,H1.8,H1.8_(Homo_sapiens)__???,,,132243.0,H1-8,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MAPGSVTSDISPSSTSTAGSSRSPESEKPGPSHGGVPPGGPSHSSL...
NP_001295191.1,NP_001295191.1,H1,H1.8,H1.8_(Homo_sapiens)__???,,,132243.0,H1-8,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MAPATAPRRAGEAKGKGPKKPSEAKEDPPNVGKVKKAAKRPAKVQK...


In [6]:
cs.data[cs.data["accession"] == "P16888.2"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
P16888.2,P16888.2,H2B,late_H2B_(Echinoidea),late_H2B_(Echinoidea),,,,,7668,Strongylocentrotus purpuratus,Echinodermata,Echinoidea,,,3697096,MPAKAQPAGKKGSKKAKAPRPSGGKKRRRRRKESYGIYIYKVLKQV...


In [7]:
cs.data.head()

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
NP_505463.1,NP_505463.1,H2A,cH2A,cH2A_(Animals),,17562014,,,6239,Caenorhabditis elegans,Nematoda,Chromadorea,,,26989147 22650316(?),MSGRGKGGKAKTGGKAKSRSSRAGLQFPVGRLHRILRKGNYAQRVG...
EEC09557.1,EEC09557.1,H2A,cH2A,cH2A_(Animals),,215500063,,,6945,Ixodes scapularis,Arthropoda,Arachnida,,,26989147 22650316(?),MSGRGKGGKVKGKSKTRSSRAGLQFPVGRIHRLLRKGNYAERVGAG...
NP_724343.1,NP_724343.1,H2A,cH2A,cH2A_(Animals),,24585673,,,7227,Drosophila melanogaster,Arthropoda,Insecta,,,26989147 22650316(?),MSGRGKGGKVKGKAKSRSNRAGLQFPVGRIHRLLRKGNYAERVGAG...
XP_001119899.1,XP_001119899.1,H2A,cH2A,cH2A_(Animals),,110764935,,,7460,Apis mellifera,Arthropoda,Insecta,,,26989147 22650316(?),MSGRGKGGKAKAKAKSRSNRAGLQFPVGRIHRLLRKGNYAERVGAG...
EDO48405.1,EDO48405.1,H2A,cH2A,cH2A_(Animals),,156227602,,,45351,Nematostella vectensis,Cnidaria,Anthozoa,,,26989147 22650316(?),MSGRGKGKAKGTKSKTRSSRAGLQFPVGRIHRHLRKGNYAERVGAG...


In [7]:
cs.data.shape

(564, 16)

In [8]:
late_h2b_accessions = [
    "P16888.2",
]

In [9]:
df = pd.DataFrame(
    {
        "accession": late_h2b_accessions,
        "type": ["H2B"] * len(late_h2b_accessions),
        "variant_group": ["late_H2B_(Echinoidea)"] * len(late_h2b_accessions),
        "variant": ["late_H2B_(Echinoidea)"] * len(late_h2b_accessions),
        "references": ["3697096"] * len(late_h2b_accessions),
    }
)
df.index = df.accession
df.shape, df.columns

((1, 5),
 Index(['accession', 'type', 'variant_group', 'variant', 'references'], dtype='object'))

In [10]:
df.head()

,accession,type,variant_group,variant,references
accession,,,,,
P16888.2,P16888.2,H2B,late_H2B_(Echinoidea),late_H2B_(Echinoidea),3697096


In [11]:
cs.data = pd.concat([cs.data, df]).fillna("")
cs.data.shape, list(cs.has_duplicates())

((565, 16), ['P16888.2'])

In [12]:
cs.data.tail()

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
NP_005314.2,NP_005314.2,H1,H1.6,H1.6_(Homo_sapiens)__???,,,3010.0,H1-6,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MSETVPAASASAGVAAMEKLPTKKRGRKPAGLISASRKVPNLSVSK...
NP_005313.1,NP_005313.1,H1,H1.5,H1.5_(Homo_sapiens)__???,,,3009.0,H1-5,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MSETAPAETATPAPVEKSPAKKKATKKAAGAGAAKRKATGPPVSEL...
NP_722575.1,NP_722575.1,H1,H1.8,H1.8_(Homo_sapiens)__???,,,132243.0,H1-8,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MAPGSVTSDISPSSTSTAGSSRSPESEKPGPSHGGVPPGGPSHSSL...
NP_001295191.1,NP_001295191.1,H1,H1.8,H1.8_(Homo_sapiens)__???,,,132243.0,H1-8,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MAPATAPRRAGEAKGKGPKKPSEAKEDPPNVGKVKKAAKRPAKVQK...
P16888.2,P16888.2,H2B,late_H2B_(Echinoidea),late_H2B_(Echinoidea),,,,,,,,,,,3697096,


In [13]:
cs.data[cs.data["variant"] == "late_H2B_(Echinoidea)"].shape

(1, 16)

In [14]:
cs.update_accession_version()
cs.data = cs.data.set_index(cs.data.accession.values)

/home/l_singh/.conda/envs/histdb_env/lib/python3.8/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(
/home/l_singh/.conda/envs/histdb_env/lib/python3.8/site-packages/Bio/GenBank/__init__.py:1143: BiopythonParserWarning: Dropping bond qualifier in feature location
  warnings.warn(
/mnt/scratch/v_kulikov/histonedb_Vladimir/CURATED_SET/curated_set_services.py:267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [15]:
cs.data[cs.data["variant"] == "late_H2B_(Echinoidea)"].shape

(1, 16)

In [16]:
curated_set.update_taxids(blank_data=True)

Fetched taxid from NCBI 7668
 changes to 7668
 changes to Strongylocentrotus purpuratus
 changes to Echinodermata
 changes to Echinoidea


In [17]:
cs.data[cs.data["variant"] == "late_H2B_(Echinoidea)"].shape

(1, 16)

In [18]:
cs.data[cs.data["variant"] == "late_H2B_(Echinoidea)"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
P16888.2,P16888.2,H2B,late_H2B_(Echinoidea),late_H2B_(Echinoidea),,,,,7668,Strongylocentrotus purpuratus,Echinodermata,Echinoidea,,,3697096,


## Updating sequences for late_H2B_(Echinoidea)

In [19]:
cs.data[cs.data['sequence'] == '']

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
P16888.2,P16888.2,H2B,late_H2B_(Echinoidea),late_H2B_(Echinoidea),,,,,7668,Strongylocentrotus purpuratus,Echinodermata,Echinoidea,,,3697096,


In [20]:
cs.update_sequence(blank_data=True)

Fetching 1 seqs


/home/l_singh/.conda/envs/histdb_env/lib/python3.8/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


Sequence for P16888.2 changes from  to MPAKAQPAGKKGSKKAKAPRPSGGKKRRRRRKESYGIYIYKVLKQVHPDTGISSRAMSIMNSFVNDVFERIAAEASRLAHYNKKSTITSREVQTVVRLLLPGELAKHAVSEGTKAVTKYTTSK
Sequences updated: 1


In [21]:
cs.data[cs.data["variant"] == "late_H2B_(Echinoidea)"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
P16888.2,P16888.2,H2B,late_H2B_(Echinoidea),late_H2B_(Echinoidea),,,,,7668,Strongylocentrotus purpuratus,Echinodermata,Echinoidea,,,3697096,MPAKAQPAGKKGSKKAKAPRPSGGKKRRRRRKESYGIYIYKVLKQV...


In [22]:
cs.save()

variant_group  self                  late_H2B
               other    late_H2B_(Echinoidea)
variant        self           late_H2B.1__???
               other    late_H2B_(Echinoidea)
Name: P16888.2, dtype: object
cp histones.csv backups/histones.csv-Mar2624081541
Previous data backuped to backups/histones.csv-Mar2624081541
Results saved to histones.csv
